In [1]:
with open("labels.txt", "r") as file:
    labels_content = file.read()
    print("Labels.txt içeriği:")
    print(labels_content)

FileNotFoundError: [Errno 2] No such file or directory: 'labels.txt'

In [ ]:
!pip install tensorflow==2.12.0 # TensorFlow'un 2.12.0 sürümünü yükleme

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.1 MB/s eta 0:00:00
 

In [ ]:
from keras.models import load_model  # Keras'ı kullanmak için TensorFlow gereklidir
from PIL import Image, ImageOps  # Pillow, resim işleme için kullanılır
import numpy as np  # NumPy, matematiksel işlemler için kullanılır

# Bilimsel gösterimi kapatarak sonuçların ondalık sayı olarak görüntülenmesini sağlar
np.set_printoptions(suppress=True)

# Eğitilmiş modeli yükle
model = load_model("keras_model.h5", compile=False)

# Sınıf etiketlerini yükle
class_names = open("labels.txt", "r").readlines()

# Keras modeline girdi olarak verilecek diziyi oluştur
# Buradaki ilk boyut (1), aynı anda kaç resmin işleneceğini belirler
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Kullanılacak görüntünün yolunu buraya ekleyin
image = Image.open("/content/cam.jpg").convert("RGB")

# Görüntüyü en az 224x224 boyutuna getirmek ve merkezden kırpmak
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# Görüntüyü bir NumPy dizisine dönüştür
image_array = np.asarray(image)

# Görüntüyü normalize et (0-1 aralığına getirme işlemi)
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Görüntüyü modelin giriş dizisine yükle
data[0] = normalized_image_array

# Modeli kullanarak tahmin yap
prediction = model.predict(data)
index = np.argmax(prediction)  # En yüksek olasılığı bulan indeks
class_name = class_names[index]  # Tahmin edilen sınıf adı
confidence_score = prediction[0][index]  # Güven skoru
class_name =  class_name[2:].strip()
# Tahmin edilen sınıfı ve güven skorunu ekrana yazdır
print("Sınıf:", class_name[2:], end="")  # İlk iki karakteri kaldırarak yazdır
print("Güven Skoru:", confidence_score)


1/1 [==============================] - 1s 983ms/step
Sınıf: m(evet)Güven Skoru: 0.88409835


In [ ]:
from google.colab import files

path = files.upload()
path = list(path.keys())[0]
image=Image.open(path)

Saving Image20250707201936.jpg to Image20250707201936.jpg


In [ ]:
def detect_geridonusum(image_path, model_path, labels_path):
    # Eğitilmiş modeli yükle
    model = load_model(model_path, compile=False)

    # Sınıf etiketlerini yükle
    class_names = open(labels_path, "r").readlines()

    # Keras modeline girdi olarak verilecek diziyi oluştur
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Kullanılacak görüntüyü aç
    image = Image.open(image_path).convert("RGB")

    # Görüntüyü en az 224x224 boyutuna getirmek ve merkezden kırpmak
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # Görüntüyü bir NumPy dizisine dönüştür
    image_array = np.asarray(image)

    # Görüntüyü normalize et (0-1 aralığına getirme işlemi)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Görüntüyü modelin giriş dizisine yükle
    data[0] = normalized_image_array

    # Modeli kullanarak tahmin yap
    prediction = model.predict(data)
    index = np.argmax(prediction)  # En yüksek olasılığı bulan indeks
    class_name = class_names[index]  # Tahmin edilen sınıf adı
    confidence_score = prediction[0][index]  # Güven skoru

    # Tahmin edilen sınıfı ve güven skorunu döndür
    return class_name[2:].strip(), confidence_score  # İlk iki karakteri kaldırarak döndür

In [ ]:
image_path = "/content/cam.jpg"
model_path = "keras_model.h5"
labels_path = "labels.txt"
class_name, confidence_score = detect_geridonusum(image_path, model_path, labels_path)

# Sonuçları yazdır
print("Sınıf:", class_name)
print("Güven Skoru:", confidence_score)


1/1 [==============================] - 1s 1s/step
Sınıf: cam(evet)
Güven Skoru: 0.88409835


In [ ]:
def suggest_geridonusum_options(class_name, confidence_score):
    """
    Kullanıcıya fotoğraftaki öğe ile ilgili bilgi seçeneklerini sunar.
    """
    # geri dönüşüm türlerine özgü ikramlar
    suggestions = {
        "cam(evet)": ["evet geri dönüşebilir.Cam silika, soda kültürü, feldspat gibi doğal kaynakların yüksek derecede eritilmesi sonucu oluşturulur. Camlar oluşurken doğal kaynakların azalmasına, petrol kaynaklarının azalmasına sebep olur. Üretimi sonucu ortaya çıkan kirlilik yüzünden de çevre kirliliği ve bazı sağlık sorunlarının oluşmasına neden olur. Doğal kaynaklarımızın giderek azaldığı, çevre sorunlarının önemli boyutlara ulaştığı günümüz çağında en az plastik, ambalaj atıklarının geri kazanılması kadar atık camların geri dönüşüm serüvenine katılması oldukça ciddi önem taşır.Cam yeniden kullanılabilir bir maddedir. Kullanılmış ve kırık cam parçalarından tekrar cam üretilmesi sıfırdan cam üretimine göre daha az enerji gerektirdiğini biliyor muydunuz? Geri dönüşüm kumbaraları atık camları biriktirebilmemiz için kolaylık sağlar. Yalnızca cam şişe veya kavanoz gibi ürünlerin kapaklarını çıkararak atmalıyız.Ülkemizde atık sahalarında bulunan çöplerin içindeki geri dönüştürülebilir veya geri kazanılabilir çöp oranı yaklaşık %15 dir. Türkiye’de cam geri kazanımı yaklaşık %40 dır ve yılda yaklaşık olarak 6500 ton atık camların geri dönüşüm tesislerinde işlenerek geri kazanıma sokulmakta, ülke ekonomisine katkı sağlanmaktadır."],
        "metal(evet)": ["evet geri dönüşebilir.Metal ürünlerin geri dönüşümündeki en önemli özelliği tekrar tekrar geri dönüştürülebilir özellikte olmalarıdır. Geri dönüşüm işlemleri, geri dönüştürülecek ürünün türüne bağlı olarak farklılık gösterir. Metal geri dönüşümleri; toplama → ayrıştırma → sınıflandırma → kırma ve ezme → parçalama → erime → temizleme → katılaştırma → yeniden kullanım için taşıma işlemi gibi farklı aşamalardan geçirilerek geri dönüştürülür. Toplama; İlk ve en önemli adımdır. Metalleri toplamak için özel olarak tasarlanmış kaplar olacak şekilde düzenlenmelidir. Hurda metal bahçeleri, metaller için toplama merkezleri olarak kullanılır. Ayrıştırma; Ayrıştırma için mıknatıslar kullanılmaktadır. Böylelikle geri dönüşümcülerin bunları karışık atık akışından kolayca ayırmaları sağlanmaktadır.  Sınıflandırma; Geri dönüşümü olanın geri dönüştürülemeyenden ayrılmasını içerir. Geri dönüştürülmüş metal kalitesinin çok önemli olduğunu belirtmek gerekir. Yüksek kalitede geri dönüştürülmüş bir ürün, ancak geri dönüşüm sürecinde kullanılan orijinal malzemeler iyi kalitede ise oluşturulabilir. Bu nedenle, Sınıflandırma işlemi sırasında sıkı kalite kontrolünü gerektirir. Kırma ve Ezme; bir sonraki adım metali sıkıştırmak veya sıkmaktır. Tüm geri dönüşüm malzemeleri, makineler kullanılarak sıkıştırılır ve ezilir, böylece konveyör bantlarında fazla yer kaplamazlar. Parçalama; Metaller, daha fazla işlem yapılmasını sağlamak için küçük parçalara veya tabakalara ayrılır. Küçük parçalar, büyük metal parçalarına kıyasla daha az enerji kullanılarak eritilebilen büyük yüzey / hacim oranına sahiptir.   Erime; Hurda metalin erimesi büyük bir fırında gerçekleşir. Her metal, kendine has özelliklerinin kaybolmayacağı şekilde bu metali eritmek için özel olarak tasarlanmış bir fırına alınır. Geri dönüştürülmüş metalin eritilmesi için gereken enerji, hammaddesinden metal yapmak için gereken enerjiyle karşılaştırıldığında daha azdır. Eritme fırının büyüklüğüne, fırına yerleştirilen metal hacmine ve fırının ısı derecesine bağlı olarak erime işlemi dakikalar veya saatler sürebilir. Temizleme; Eritme işlemi tamamlandıktan sonra, bir sonraki adım saflaştırma yani temizleme işlemidir. Metallerin saflaştırılması, nihai ürünün safsızlıktan ve yüksek kalitede olmasını sağlamak için yapılır. Elektroliz, bazı metalleri saflaştırma yöntemlerinden biridir. Diğer metaller, metalleri diğer geri dönüştürülebilir maddelerden ayıran güçlü manyetik sistemler altına girer. Bugün metal türüne bağlı olarak farklı arıtma-temizleme yöntemleri vardır. Katılaştırma; Saflaştırma işleminden sonra, erimiş metal daha sonra taşıyıcı kayış tarafından soğutulduğu ve katılaştırıldığı bir soğutma odasına taşınır. Bu aşamada, hurda metal tekrar kullanılabilecek sağlam bir metale dönüştürülür. Daha sonra yoğunluğunu ve diğer özelliklerini elde etmek için erimiş metale başka kimyasallar eklenir. Soğutma aşamasında, farklı şekillerde ve boyutlarda metaller yapılır ve tasarlanır. Yeniden kullanım için taşıma işlemi Çubuklar tasarlandıktan ve yapıldıktan sonra, nihai ürün daha sonra farklı fabrikalara ve metale ihtiyaç duyan insanlara taşınmaya hazır boyutlarına ve şekillerine bağlı olarak paketlenir. Bundan sonra, döngü tekrar başlar."],
        "kagit(evet)": ["evet geri dönüşebilir.Kâğıt geri dönüşüm tesislerinde de bu deneydekine benzer işlemler yapılır. Öncelikle atık kâğıtlar, farklı yerlerdeki kâğıt geri dönüşüm kutularında toplanır ve ardından hep birlikte büyük bir geri dönüşüm konteynerine konur. Daha sonra konteynerin içindeki atık kâğıtlar, bir toplama aracıyla kâğıt geri dönüşüm tesisine taşınır. Geri dönüşüm tesisine gelen kâğıtlar cinslerine göre ayrılır. Örneğin karton kâğıtlar, parlak dergi kâğıtları ve beyaz kâğıtlar birbirinden ayrıştırılır. Ayrıştırılan kâğıtlar, kâğıt öğütme makinesinde küçük parçalar hâlinde öğütülür. Öğütülen kâğıtlar arıtma, ağartma ve renk silme işlemleriyle yazı, renk ve desenlerinden arındırılır. Hamur hâline getirilen karışım, atık kâğıtların liflerine ayrılması için suda bekletilir. %99,5’i su olan karışımda kâğıttaki mürekkebin tamamen çıkarılması için sodyum karbonat ve sodyum hidroksit de kullanılır. Bu şekilde kâğıt parçaları hem liflerine ayrılır hem de temizlenir. Ardından kâğıt yapma makineleri tarafından baskı uygulanarak hamurun içerisindeki su tamamen çıkarılır. Son aşamada ise kuruyan kâğıt, büyük rulolara sarılır.Geri dönüştürülen atık kâğıtlar ise daha çok gazete ve mukavva yapımında kullanılır.Siz de atık kâğıtlarınızı belediyelerin geri dönüşüm kutularına atabilir ya da çöp torbasında biriktirip bir geri dönüşüm tesisine iletebilirsiniz. "],
        "motor yagi(evet?)":["evet geri dönüşebilir"],
        "karton(evet)": ["evet geri dönüşebilir.Geri dönüştürme kutusuna attığınız her içecek kartonu yeni bir başlangıca doğru ilerlemeye yardımcı olur ve değerli kaynakların kullanılarak çöplerden uzak tutulmasını sağlar. Yapmanız gereken tek şey, kapak da dahil tüm paketi geri dönüşüm kutusuna atmadan önce kartonları boşaltmak ve düzleştirmektir."],
        "plastik(evet)": ["evet geri dönüşebilir.Plastik atıklar lisanslı plastik atık geri dönüşüm tesisi bünyesine gönderilir. Plastik atıklar geri dönüşüm işlemlerine tabi tutulmadan önce cinslerine göre ayrılır. Cinslerine göre ayrıştırılan geri dönüştürülebilir plastik atıklar, kırma makinelerine atılır ve kırılır. Kırıldıktan sonra küçük parçalara ayrılır."],
        "naylon poset(hayir)": ["hayır geri dönüşmez. olabildiğince az kullanın"],
        "florasan lambalar(hayir)": ["hayır geri dönüşmez. olabildigince az kullanın"]
    }

    # geri dönüşen türüne bağlı öneriler
    if class_name.lower() in suggestions:
        options = suggestions[class_name.lower()]
    else:
        options = ["tanımıyom malesef"]

    print(f"Fotoğraftaki öğe: {class_name}")
    print(f"Modelin güven skoru: {confidence_score * 100:.2f}%")
    print("geri dönüşüp dönüşmemesi:")
    for option in options:
        print(f"- {option}")


In [ ]:
suggest_geridonusum_options(class_name, confidence_score)

Fotoğraftaki öğe: cam(evet)
Modelin güven skoru: 88.41%
geri dönüşüp dönüşmemesi:
- evet geri dönüşebilir.Cam silika, soda kültürü, feldspat gibi doğal kaynakların yüksek derecede eritilmesi sonucu oluşturulur. Camlar oluşurken doğal kaynakların azalmasına, petrol kaynaklarının azalmasına sebep olur. Üretimi sonucu ortaya çıkan kirlilik yüzünden de çevre kirliliği ve bazı sağlık sorunlarının oluşmasına neden olur. Doğal kaynaklarımızın giderek azaldığı, çevre sorunlarının önemli boyutlara ulaştığı günümüz çağında en az plastik, ambalaj atıklarının geri kazanılması kadar atık camların geri dönüşüm serüvenine katılması oldukça ciddi önem taşır.Cam yeniden kullanılabilir bir maddedir. Kullanılmış ve kırık cam parçalarından tekrar cam üretilmesi sıfırdan cam üretimine göre daha az enerji gerektirdiğini biliyor muydunuz? Geri dönüşüm kumbaraları atık camları biriktirebilmemiz için kolaylık sağlar. Yalnızca cam şişe veya kavanoz gibi ürünlerin kapaklarını çıkararak atmalıyız.Ülkemizde atık s